<a href="https://colab.research.google.com/github/HanzlaJavaid/Brain-Computing-Interface/blob/main/Conditional_GAN_EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from __future__ import print_function, division
import pandas as pd
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt

import numpy as np

class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 10
        self.img_cols = 5
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 2
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        #(X_train, y_train), (_, _) = mnist.load_data()

        df = pd.read_csv("tutorial_tea.csv")
        scaler = MinMaxScaler()
        X = scaler.fit_transform(df.values[:,1:-1])
        Y = df.values[:,-1]
        Xs = np.reshape(X,(-1,10,5,1))
        print(Xs.shape)
        Ys = to_categorical(Y)
        X_train = Xs
        y_train = Ys

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 1, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 1, 2
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 2).reshape(-1, 1)
        print(noise)
        print(sampled_labels)
        gen_imgs = self.generator.predict((noise, sampled_labels))
        print(gen_imgs)
        print(gen_imgs.shape)


In [9]:
cgan = CGAN()
cgan.train(epochs=20000, batch_size=32, sample_interval=200)

Streaming output truncated to the last 5000 lines.

  [[-0.99932694]
   [-0.996435  ]
   [-0.99746996]
   [-0.9975257 ]
   [-0.99896735]]

  [[-0.997395  ]
   [-0.9974985 ]
   [-0.9957736 ]
   [-0.9986721 ]
   [-0.9988043 ]]

  [[-0.99626744]
   [-0.9983392 ]
   [-0.99407864]
   [-0.9979878 ]
   [-0.9993448 ]]

  [[-0.9980727 ]
   [-0.9975053 ]
   [-0.99579996]
   [-0.99612844]
   [-0.9983985 ]]

  [[-0.99974245]
   [-0.9975112 ]
   [-0.9946194 ]
   [-0.9978057 ]
   [-0.9948367 ]]

  [[-0.99636376]
   [-0.9964822 ]
   [-0.99614424]
   [-0.99449223]
   [-0.9941299 ]]]


 [[[-0.9969914 ]
   [-0.9947854 ]
   [-0.99145925]
   [-0.99317086]
   [-0.997687  ]]

  [[-0.9978911 ]
   [-0.99377936]
   [-0.99479234]
   [-0.99833804]
   [-0.9963433 ]]

  [[-0.99552214]
   [-0.9985605 ]
   [-0.9969696 ]
   [-0.9879526 ]
   [-0.9966271 ]]

  [[-0.9976472 ]
   [-0.9989527 ]
   [-0.99856126]
   [-0.9940596 ]
   [-0.9963572 ]]

  [[-0.998603  ]
   [-0.993511  ]
   [-0.9978268 ]
   [-0.9764059 ]
   [-0.9

KeyboardInterrupt: ignored